In [1]:
import cv2
import numpy as np
import sys
sys.path.insert(0, 'D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib')
import color_deconvolution as cd
import ImageObject as im_o
from os.path import join
from skimage.measure import regionprops
import scipy.ndimage as ndi
from os.path import isfile
from os import listdir
import scipy.ndimage as ndi


#validation_prefixes = ['H2128-03_CD30_0_7_5', 'K1147-04_CD30_0_27_7', 'K1440-11_CD30_0_7_18', 'K1164-11_CD30_0_3_8', 'K879-91_CD30_0_15_19']
validation_prefixes = ['12727-05_CD30_0_15_14', 'K46-12_CD30_0_6_8', 'K46-12_CD30_0_8_2', 'H2128-03_CD30_0_7_5', 'K1147-04_CD30_0_27_7', 'K1440-11_CD30_0_7_18', 'K1164-11_CD30_0_3_8', 'K879-91_CD30_0_15_19', 'K1149-11_CD30_0_26_18', 'K879-91_CD30_0_4_11']
val_im_folder = 'D:/old_stuff/new_stuff/slightly_better_training/validation_images'
val_an_folder = 'D:/old_stuff/new_stuff/slightly_better_training/validation_annotations'

def get_good_means(ims, ans):
    val_arrs = [[], [], [], [], [], [], [], []]
    means = np.zeros((8))
    for im, an in list(zip(ims, ans)):
        for i in range(1,9):
            mask = an == i
            vals = np.extract(mask, im[:,:,2])
            val_arrs[i-1] += list(vals)
    for i in range(8):
        if (len(val_arrs[i]) > 0):
            mean = np.mean(val_arrs[i])
            means[i] = mean
        else:
            means[i] = means[i-1] + 5.
    print('number of vals in each class is')
    for ll in val_arrs:
        print(len(ll))
    print('means are')
    print(means)
    return means

def transform_mask(im, mask, good_means):
    out = np.zeros(mask.shape)
    cell_im = (mask == 1) + (mask == 2)
    art_im = mask == 5
    uns_im = mask == 7
    lbl, num = ndi.label(cell_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[0]), np.linalg.norm(m_val-good_means[1]), np.linalg.norm(m_val-good_means[2])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    lbl, num = ndi.label(art_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[3]), np.linalg.norm(m_val-good_means[4]), np.linalg.norm(m_val-good_means[5])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    lbl, num = ndi.label(uns_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[6]), np.linalg.norm(m_val-good_means[7])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    return out
            

def make_training_images(folder_ims, folder_ans, out_folder_ims, out_folder_ans, index=0):
    im_names = [f for f in listdir(folder_ims) if ('.jpg' in f)]
    an_names = [f for f in listdir(folder_ans) if ('.tif' in f)]
    im_prefixes = []
    im_prefixes_good = []
    an_prefixes = []
    an_prefixes_good = []
    for im_name in im_names:
        if ('cut' in im_name):
            im_prefixes.append(im_name[:im_name.index('cut')])
            im_prefixes_good.append(im_name[:im_name.index('cut')])
        else:
            im_prefixes.append(im_name[:im_name.index('.jpg')])
    for an_name in an_names:
        if ('_multi' in an_name):
            an_prefixes.append(an_name[:an_name.index('_multi')])
            an_prefixes_good.append(an_name[:an_name.index('_multi')])
        elif ('_an' in an_name):
            an_prefixes.append(an_name[:an_name.index('_an')])
    ims = []
    ans = []
    good_ims = []
    good_ans = []
    print('Fetching data for good means')
    for im_pref in im_prefixes_good:
        im_index = im_prefixes.index(im_pref)
        an_index = an_prefixes.index(im_pref)
        im = cv2.imread(join(folder_ims, im_names[im_index]))
        im_object = im_o.ImageObject(numpy_array = cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        cd_im, hem = cd.colour_deconvolution(im_object, path='D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib/color_deconvolution_lib/cd30_hematoxylin_rest.csv')
        im_seg = np.zeros(im.shape)
        im_seg[:,:,0] = hem.get_numpy_array()
        im_seg[:,:,2] = cd_im.get_numpy_array()
        im = im_seg
        an = cv2.imread(join(folder_ans, an_names[an_index]), 0)
        good_ims.append(im)
        good_ans.append(an)
    print('Got images for good means')
    good_means = get_good_means(good_ims, good_ans)
    index = 0
    
    for im_pref in im_prefixes:
        print('Processing ' + im_pref)
        im_index = im_prefixes.index(im_pref)
        an_index = an_prefixes.index(im_pref)
        im = cv2.imread(join(folder_ims, im_names[im_index]))
        im_object = im_o.ImageObject(numpy_array = cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        cd_im, hem = cd.colour_deconvolution(im_object, path='D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib/color_deconvolution_lib/cd30_hematoxylin_rest.csv')
        im_seg = np.zeros(im.shape)
        im_seg[:,:,0] = hem.get_numpy_array()
        im_seg[:,:,2] = cd_im.get_numpy_array()
        im = im_seg
        an = cv2.imread(join(folder_ans, an_names[an_index]), 0)
        if (not (im_pref in im_prefixes_good)):
            print('Fixing image')
            an = transform_mask(im, an, good_means)
        print('Unique values are')
        print(np.unique(an))
        for i in range(0, im.shape[0], 256):
            for k in range(0, im.shape[1], 256):
                if (((i + 512) <= im.shape[0]) and ((k+512) <= im.shape[1])):
                    im_part = np.zeros((512,512,3))
                    im_part = np.copy(im[i:i+512,k:k+512])
                    an_part = np.zeros((512, 512))
                    an_part = np.copy(an[i:i+512,k:k+512])
                    im_name = str(index) + '.tif'
                    if im_pref in validation_prefixes:
                        cv2.imwrite(join(val_im_folder, im_name), im_part)
                        cv2.imwrite(join(val_an_folder, im_name), an_part)
                    else:
                        cv2.imwrite(join(out_folder_ims, im_name), im_part)
                        cv2.imwrite(join(out_folder_ans, im_name), an_part)
                    index += 1
    print('Finished writing all images')
    return

folder = 'D:/old_stuff/new_stuff/slightly_better_training'
make_training_images(join(folder, 'big_images'), join(folder, 'big_annotations'), join(folder, 'images'), join(folder, 'annotations'))

Fetching data for good means
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finished
[COLOR_DECONVOLUTION] Separation of CD30 and Hematoxylin started
[COLOR_DECONVOLUTION] Color matrix loaded.
[COLOR_DECONVOLUTION] Finishe